In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import copy
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
data = pd.read_csv("Train_Data.csv")
from sklearn.model_selection import train_test_split
trainData, testData = train_test_split(data, test_size=0.2, random_state=42)

In [3]:
print(trainData['price range'])

3994    4
423     2
2991    3
1221    1
506     1
       ..
1130    1
1294    4
860     4
3507    4
3174    1
Name: price range, Length: 3200, dtype: int64


In [4]:
class ResidualBlock(nn.Module):
    expansion: int = 1

    def __init__(self, inchannel, outchannel, stride=1):
        super(ResidualBlock, self).__init__()
        self.left = nn.Sequential(
            nn.Conv2d(inchannel, outchannel, kernel_size=3,
                      stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(outchannel),
            nn.ReLU(inplace=True),
            nn.Conv2d(outchannel, outchannel, kernel_size=3,
                      stride=1, padding=1, bias=False),
            nn.BatchNorm2d(outchannel)
        )
        self.shortcut = nn.Sequential()
        if stride != 1 or inchannel != outchannel * self.expansion:
            self.shortcut = nn.Sequential(
                nn.Conv2d(inchannel, outchannel * self.expansion, kernel_size=1,
                          stride=stride, bias=False),
                nn.BatchNorm2d(outchannel * self.expansion)
            )

    def forward(self, x):
        out = self.left(x)
        out += self.shortcut(x)#open shortcut connection
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, ResidualBlock, num_classes=5):
        super(ResNet, self).__init__()
        self.inchannel = 3
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 3, kernel_size=2, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(3),
            nn.ReLU(),
        )
        self.layer1 = self.make_layer(ResidualBlock, 3,  2, stride=1)
        self.layer2 = self.make_layer(ResidualBlock, 6, 2, stride=2)
        self.layer3 = self.make_layer(ResidualBlock, 12, 2, stride=2)
        self.layer4 = self.make_layer(ResidualBlock, 24, 2, stride=2)
        self.fc1 = nn.Linear(24, 12)
        self.fc2 = nn.Linear(12, num_classes)
        self.dropout = nn.Dropout(p=0.5)

    def make_layer(self, block, channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)  # strides=[1,1]
        layers = []
        for stride in strides:
            layers.append(block(self.inchannel, channels, stride))
            self.inchannel = channels
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        #out = self.dropout(out)
        out = self.layer1(out)
        #out = self.dropout(out)
        out = self.layer2(out)
       # out = self.dropout(out)
        out = self.layer3(out)
        #out = self.dropout(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out,1)
        out = out.view(out.size(0), -1)
        out = F.relu(self.fc1(out))
        out = self.dropout(out)
        out = self.fc2(out)
        return out
def ResNet18():
    return ResNet(ResidualBlock)

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=(2,2),stride=1,padding=1)
        self.pool1 =nn.MaxPool2d(2,2)
        #self.pool2  = nn.Maxpool2d(1,2)
        #self.conv2 = nn.Conv2d(10, 20, kernel_size=(1,1))
        self.flatten = nn.Flatten(start_dim=1, end_dim=-1)
        self.fc1 = nn.Linear(24,12)
        #self.fc2 = nn.Linear(16,8)
        self.fc2 = nn.Linear(12,5)
        #self.dropout = nn.Dropout(p=0.5)
    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        #x = self.pool2(F.relu(self.conv2(x)))
        #x = x.view(-1, 16 * 1 *1 )
        x = self.flatten(x)
        #x = self.dropout(x)
        x = F.relu(self.fc1(x))
        #x = self.dropout(x)
        #x = F.softmax(self.fc2(x))
        #x = self.dropout(x)
        x = self.fc2(x)
        return x

In [5]:
def dirichlet_partition(training_data, testing_data, alpha, user_num):
    idxs_train = np.arange(len(training_data))
    idxs_valid = np.arange(len(testing_data))

    if hasattr(training_data, 'price range'):
        labels_train = training_data['price range']
        labels_valid = testing_data['price range']
    elif hasattr(training_data, 'img_label'):
        labels_train = training_data.img_label
        labels_valid = testing_data.img_label

    idxs_labels_train = np.vstack((idxs_train, labels_train))
    # print("idxs_labels_train 1: ", idxs_labels_train)
    idxs_labels_train = idxs_labels_train[:, idxs_labels_train[1,:].argsort()]
    # print("idxs_labels_train 2: ", idxs_labels_train)
    idxs_labels_valid = np.vstack((idxs_valid, labels_valid))
    idxs_labels_valid = idxs_labels_valid[:, idxs_labels_valid[1,:].argsort()]

    labels = np.unique(labels_train, axis=0)

    data_train_dict = data_organize(idxs_labels_train, labels)
    # print("data_train_dict[0]: ", data_train_dict[0])
    # print("data_train_dict[9]: ", data_train_dict[9])
    data_valid_dict = data_organize(idxs_labels_valid, labels)

    data_partition_profile_train = {}
    data_partition_profile_valid = {}


    for i in range(user_num):
        data_partition_profile_train[i] = []
        data_partition_profile_valid[i] = []

    ## Distribute rest data
    for label in data_train_dict:
        proportions = np.random.dirichlet(np.repeat(alpha, user_num))
        proportions_train = len(data_train_dict[label])*proportions
        proportions_valid = len(data_valid_dict[label]) * proportions

        for user in data_partition_profile_train:

            data_partition_profile_train[user]   \
                = set.union(set(np.random.choice(data_train_dict[label], int(proportions_train[user]) , replace = False)), data_partition_profile_train[user])
            data_train_dict[label] = list(set(data_train_dict[label])-data_partition_profile_train[user])


            data_partition_profile_valid[user] = set.union(set(
                np.random.choice(data_valid_dict[label], int(proportions_valid[user]),
                                 replace=False)), data_partition_profile_valid[user])
            data_valid_dict[label] = list(set(data_valid_dict[label]) - data_partition_profile_valid[user])


        while len(data_train_dict[label]) != 0:
            rest_data = data_train_dict[label][0]
            user = np.random.randint(0, user_num)
            data_partition_profile_train[user].add(rest_data)
            data_train_dict[label].remove(rest_data)

        while len(data_valid_dict[label]) != 0:
            rest_data = data_valid_dict[label][0]
            user = np.random.randint(0, user_num)
            data_partition_profile_valid[user].add(rest_data)
            data_valid_dict[label].remove(rest_data)

    for user in data_partition_profile_train:
        data_partition_profile_train[user] = list(data_partition_profile_train[user])
        data_partition_profile_valid[user] = list(data_partition_profile_valid[user])
        np.random.shuffle(data_partition_profile_train[user])
        np.random.shuffle(data_partition_profile_valid[user])

    return data_partition_profile_train, data_partition_profile_valid


def data_organize(idxs_labels, labels):
    data_dict = {}

    labels = np.unique(labels, axis=0)
    for one in labels:
        data_dict[one] = []

    for i in range(len(idxs_labels[1, :])):
        data_dict[idxs_labels[1, i]].append(idxs_labels[0, i])
    return data_dict

In [6]:
user_num = 4
alpha = 0.5
train_index, test_index = dirichlet_partition(trainData, testData, alpha=alpha, user_num=user_num)
print("train_index[0]：", train_index[0])
print("train_index[1]：", train_index[1])


train_index[0]： [3196, 1903, 2602, 790, 3019, 366, 1121, 2151, 3045, 2761, 2143, 429, 2213, 2828, 2815, 1246, 1002, 2625, 2073, 2389, 307, 2450, 1862, 2412, 2579, 2685, 3157, 2923, 2077, 984, 2559, 163, 2593, 2091, 1077, 1554, 2590, 2558, 2939, 1043, 2270, 1345, 623, 1650, 1041, 2565, 1811, 96, 2817, 2365, 181, 75, 1538, 1599, 1249, 106, 586, 278, 430, 1151, 3039, 850, 141, 337, 2763, 1780, 543, 1214, 471, 316, 1908, 1268, 1283, 1669, 3068, 1819, 8, 156, 793, 2300, 2573, 1425, 1147, 924, 1936, 2826, 1229, 2706, 1937, 770, 250, 1556, 1134, 2141, 2194, 1356, 2388, 2234, 1529, 2122, 1078, 2529, 426, 646, 933, 2320, 1612, 1563, 1536, 2186, 2536, 121, 1372, 256, 1648, 2037, 1462, 299, 2836, 928, 1899, 1967, 2459, 2264, 196, 2628, 411, 1274, 1932, 2837, 3077, 1726, 1052, 2608, 340, 1508, 2714, 1288, 371, 1255, 1641, 2657, 2702, 621, 2423, 2525, 18, 1939, 1097, 1140, 1296, 1501, 61, 249, 3084, 1740, 2516, 3113, 2493, 1009, 913, 2279, 2313, 1209, 2523, 2781, 867, 2638, 938, 1399, 730, 918, 251

In [7]:
class DatasetSplit(Dataset):
    """An abstract Dataset class wrapped around Pytorch Dataset class.
    """

    def __init__(self, dataset, idxs):
        self.dataset = dataset
        self.idxs = [int(i) for i in idxs]

    def __len__(self):
        return len(self.idxs)

    def __getitem__(self, item):
        image, label = self.dataset[self.idxs[item]]
        return torch.tensor(image), torch.tensor(label)

train_data_list = []
for user_index in range(user_num):
    train_data_list.append(DatasetSplit(trainData, train_index[user_index]))


In [8]:
print(train_data_list[0])

In [10]:
def local_trainer(dataset, model, global_round, device, local_epoch, batchsize):

    dataloader = DataLoader(dataset, batch_size=batchsize, shuffle=True)
    criterion = nn.CrossEntropyLoss().to(device)
    model.train()
    epoch_loss = []
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    for iter in range(local_epoch):
        batch_loss = []
        for batch_idx, (images, labels) in enumerate(dataloader):
            images, labels = images.to(device), labels.to(device)
            model.zero_grad()
            logits = model(images)
            loss = criterion(logits, labels)
            loss.backward()
            optimizer.step()

            if batch_idx % 10 == 0:
                print('| Global Round : {} | Local Epoch : {} | [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    global_round, iter, batch_idx * len(images),
                    len(dataloader.dataset),
                    100. * batch_idx / len(dataloader), loss.item()))
            batch_loss.append(loss.item())
        epoch_loss.append(sum(batch_loss)/len(batch_loss))
    return model.state_dict(), sum(epoch_loss) / len(epoch_loss)

In [9]:
def local_trainer(dataset, net, global_round, device, local_epoch, batchsize,testloader):
    train_losses = []
    test_losses = []
    trainloader = DataLoader(dataset, batch_size=batchsize, shuffle=True)
    testloader = DataLoader(testloader, batch_size=batch_size, shuffle=True)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(net.parameters(), lr=0.01)
    for epoch in range(local_epoch):
        running_loss = 0.0
        total=0
        correct=0
        for i, data in enumerate(trainloader):
            inputs, labels = data
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            #optimizer.zero_grad()
            optimizer.step()
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            #print(predicted)
            #print(labels)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        train_loss = running_loss / len(trainloader)
        train_losses.append(train_loss)
        print(f'train accuracy = {100 * correct / total:.2f}%')
        # Test the model on the test set
        # Test the model on the test set
        correct = 0
        total = 0
        running_loss = 0.0
        with torch.no_grad():
            for data in testloader:
                images, labels = data
                outputs = net(images)
                loss = criterion(outputs, labels)
                running_loss += loss.item()
                #print(outputs)
                _, predicted = torch.max(outputs.data, 1)
                #print(predicted)
                #print(labels)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        test_loss = running_loss / len(testloader)
        test_losses.append(test_loss)

In [12]:
def inference(model, testloader):
    """ Returns the inference accuracy and loss.
    """
    model.eval()
    criterion = nn.CrossEntropyLoss().to(device)
    loss, total, correct = 0.0, 0.0, 0.0
    for batch_idx, (images, labels) in enumerate(testloader):
        images, labels = images.to(device), labels.to(device)

        # Inference
        outputs = model(images)
        batch_loss = criterion(outputs, labels)
        loss += batch_loss.item()

        # Prediction
        _, pred_labels = torch.max(outputs, 1)
        pred_labels = pred_labels.view(-1)
        correct += torch.sum(torch.eq(pred_labels, labels)).item()
        total += len(labels)
    loss /= batch_idx
    accuracy = correct/total
    return accuracy, loss

In [10]:
def average_weights(w):
    """
    Returns the average of the weights.
    """
    w_avg = copy.deepcopy(w[0])
    for key in w_avg.keys():
        for i in range(1, len(w)):
            w_avg[key] += w[i][key]
        w_avg[key] = torch.div(w_avg[key].float(), len(w))
    return w_avg

In [11]:
batch_size = 96

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
global_model = Net().to(device)


In [12]:
global_rounds = 5
local_epochs = 5
for round_idx in range(global_rounds):
    local_weights = []
    local_losses = []
    global_acc = []

    for user_index in range(user_num):
        model_weights, loss = local_trainer(train_data_list[user_index], copy.deepcopy(global_model), round_idx, device, local_epochs, batch_size,testData)
        local_weights.append(copy.deepcopy(model_weights))
        local_losses.append(loss)

    global_weight = average_weights(local_weights)
    global_model.load_state_dict(global_weight)

KeyError: 2855